Tisya Sharma 002903942

Professor Miguel Fuentes-Cabrera

15 October, 2025

DS4400: Machine Learning and Data Mining

Homework 4

In [1]:
import autograd.numpy as np

Problem 1

In [2]:
X = np.array([4, 7, 9, 11, 15], dtype=float)
Y = np.array([-26, -25, -23, -22, -20], dtype=float)

print("NumPy empirical X:", round(float(np.var(X, ddof=1)), 6))
print("NumPy population X:", round(float(np.var(X, ddof=0)), 6))
print("NumPy empirical Y:", round(float(np.var(Y, ddof=1)), 6))
print("NumPy population Y:", round(float(np.var(Y, ddof=0)), 6))

NumPy empirical X: 17.2
NumPy population X: 13.76
NumPy empirical Y: 5.7
NumPy population Y: 4.56


Problem 2

In [3]:
# Ridge regression with gradient descent

lam = 0.2
X = np.array([[1,-2,1,0],
              [-2,-1,2,1],
              [1, 2,1,-1]], dtype=float)
y = np.array([[2],[3],[-1]], dtype=float)
w0 = np.array([[-3],[0],[2],[1]], dtype=float)

def ridge_loss(w):
    r = np.dot(X, w) - y
    return float(np.sum(r**2) + lam*np.sum(w**2))

def grad_ridge(w):
    return 2*np.dot(X.T, (np.dot(X, w) - y)) + 2*lam*w

# picked an lr that converges
lr = 0.05
max_iter = 100000
tol = 1e-10

w = w0.copy()
prev = 1e18
for t in range(1, max_iter+1):
    L = ridge_loss(w)
    if abs(prev - L) < tol:
        break
    prev = L
    w = w - lr * grad_ridge(w)

print("Converged in", t, "iterations")
print("Final loss:", round(float(L), 12))
print("Final w:")
for i, name in enumerate(["w1","w2","w3","w4"]):
    print(name, "=", round(float(w[i,0]), 9))

w_gd = w

Converged in 364 iterations
Final loss: 0.246807262619
Final w:
w1 = -0.221556582
w2 = -0.684796388
w3 = 0.801512867
w4 = 0.217913067


In [4]:
# check singularity of X^T X
XT_X = np.dot(X.T, X)
print("rank(X^T X) =", np.linalg.matrix_rank(XT_X), " out of 4")

# ridge closed form 
w_cf_ridge = np.dot(np.linalg.inv(XT_X + lam*np.eye(X.shape[1])), np.dot(X.T, y))
print("Ridge closed-form w:")
for i, name in enumerate(["w1","w2","w3","w4"]):
    print(name, "=", round(float(w_cf_ridge[i,0]), 9))

# compare to GD
diff = np.linalg.norm(w_cf_ridge - w_gd)
print("||w_ridge_cf - w_gd||_2 =", round(float(diff), 12))

rank(X^T X) = 3  out of 4
Ridge closed-form w:
w1 = -0.22151398
w2 = -0.684772044
w3 = 0.801518953
w4 = 0.218010443
||w_ridge_cf - w_gd||_2 = 0.000109209092


The plain OLS formula fails because (X^T X) can’t be inverted since it is singular. That happens here since there are more weights than data points (p>n), so (X^T X) is not full rank. Ridge fixes this by adding (lambda I), which makes the matrix invertible, gives a unique and stable solution, and helps prevent overfitting, especially when features outnumber samples or are highly collinear.


Problem 3 (a)

In [5]:
import pandas as pd
import autograd.numpy as np

# read
df = pd.read_csv("recipes_df.csv")

# target and features
y = df["Protein"].astype(float)
X_df = df.drop(columns=["Protein"])

# split by dtype (only 'Recipe' is categorical here)
cat_cols = ["Recipe"] if "Recipe" in X_df.columns else []
num_cols = [c for c in X_df.columns if c not in cat_cols]  # ['Sodium','Calcium','Potassium'] in your file

# one-hot encode categoricals (0/1) and keep numeric raw (no scaling here)
X_cats = pd.get_dummies(X_df[cat_cols], drop_first=True, dtype=int) if len(cat_cols) else pd.DataFrame(index=X_df.index)
X_num  = X_df[num_cols].copy()

print("num cols:", num_cols, "| cat cols:", cat_cols, "| one-hot dims:", X_cats.shape[1])

num cols: ['Sodium', 'Calcium', 'Potassium'] | cat cols: ['Recipe'] | one-hot dims: 1


Problem 3 (b)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# 60/20/20 split
Xn_tmp, Xn_test, y_tmp, y_test, Xc_tmp, Xc_test = train_test_split(
    X_num, y, X_cats, test_size=0.20, random_state=0
)
Xn_train, Xn_val, y_train_s, y_val_s, Xc_train, Xc_val = train_test_split(
    Xn_tmp, y_tmp, Xc_tmp, test_size=0.25, random_state=0
)

# scalers / poly on train only
num_scaler = StandardScaler().fit(Xn_train.values)
poly2 = PolynomialFeatures(degree=2, include_bias=False).fit(num_scaler.transform(Xn_train.values))

def make_Phi(numeric_df, cats_df):
    A = num_scaler.transform(numeric_df.values)      # scaled numeric
    E = poly2.transform(A)                           # [num, num^2, crosses]  (NO bias here)
    D = cats_df.values.astype(float) if cats_df.shape[1] else np.zeros((len(cats_df), 0))
    M = np.column_stack([E, D])                      # concat
    Phi = pd.DataFrame(M)
    Phi.insert(0, "intercept", 1.0)                  # single explicit intercept
    return Phi

Phi_train = make_Phi(Xn_train, Xc_train)
Phi_val   = make_Phi(Xn_val,   Xc_val)
Phi_test  = make_Phi(Xn_test,  Xc_test)

# y to column vectors
y_train = y_train_s.values.reshape(-1,1).astype(float)
y_val   = y_val_s.values.reshape(-1,1).astype(float)
y_test  = y_test.values.reshape(-1,1).astype(float)

print("Phi shapes:", Phi_train.shape, Phi_val.shape, Phi_test.shape)

Phi shapes: (20, 11) (7, 11) (7, 11)


Problem 3 (c)

In [7]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error

def mse(Phi, y, w):
    r = np.dot(Phi, w) - y
    return float(np.mean(r**2))

# mask so we do not regularize the intercept (first column)
no_penalty = np.ones((Phi_train.shape[1], 1))
no_penalty[0,0] = 0.0

# simple GD solvers
def gd_ridge(Phi, y, lam, lr=0.02, niter=8000, tol=1e-8):
    n, p = Phi.shape
    w = np.zeros((p,1))
    for t in range(niter):
        r = np.dot(Phi, w) - y
        grad = (2.0/n) * np.dot(Phi.T, r)
        grad[0,0] = 0.0
        grad += 2.0 * lam * np.vstack([[[0.0]], w[1:]])
        w_new = w - lr * grad
        if not np.isfinite(w_new).all(): lr *= 0.5; continue
        if np.linalg.norm(w_new - w) < tol: w = w_new; break
        w = w_new
    return w

def gd_lasso(Phi, y, lam, lr=0.02, niter=20000, tol=1e-8):
    n, p = Phi.shape
    w = np.zeros((p,1))
    for t in range(niter):
        r = np.dot(Phi, w) - y
        grad = (2.0/n) * np.dot(Phi.T, r)
        grad[0,0] = 0.0
        w_mid = w - lr * grad
        w_new = w_mid.copy()
        w_new[1:] = np.sign(w_mid[1:]) * np.maximum(np.abs(w_mid[1:]) - lr*lam, 0.0)
        if not np.isfinite(w_new).all(): lr *= 0.5; continue
        if np.linalg.norm(w_new - w) < tol: w = w_new; break
        w = w_new
    return w

def gd_elastic(Phi, y, lam, l1_ratio=0.5, lr=0.02, niter=20000, tol=1e-8):
    n, p = Phi.shape
    a = l1_ratio; b = 1.0 - a
    w = np.zeros((p,1))
    for t in range(niter):
        r = np.dot(Phi, w) - y
        grad = (2.0/n) * np.dot(Phi.T, r)
        grad[0,0] = 0.0
        grad += 2.0 * lam * b * np.vstack([[[0.0]], w[1:]])        # ridge part
        w_mid = w - lr * grad
        thr = lr * lam * a
        w_new = w_mid.copy()
        w_new[1:] = np.sign(w_mid[1:]) * np.maximum(np.abs(w_mid[1:]) - thr, 0.0)
        if not np.isfinite(w_new).all(): lr *= 0.5; continue
        if np.linalg.norm(w_new - w) < tol: w = w_new; break
        w = w_new
    return w

# grid of lambdas
lambda_list_l1 = [0.01, 0.1, 1.0, 10.0]      # for lasso/elastic
lambda_list_l2 = [0.0, 0.01, 0.1, 1.0, 10.0] # for ridge

# Lasso
best_lam_lasso, best_val_lasso, best_w_lasso = None, 1e18, None
for lam in lambda_list_l1:
    w = gd_lasso(Phi_train.values, y_train, lam, lr=0.02, niter=20000)
    v = mse(Phi_val.values, y_val, w)
    if v < best_val_lasso: best_val_lasso, best_lam_lasso, best_w_lasso = v, lam, w
print("Lasso (GD): best λ =", best_lam_lasso, " | val MSE =", round(best_val_lasso, 6))

# Ridge
best_lam_ridge, best_val_ridge, best_w_ridge = None, 1e18, None
for lam in lambda_list_l2:
    w = gd_ridge(Phi_train.values, y_train, lam, lr=0.02, niter=8000)
    v = mse(Phi_val.values, y_val, w)
    if v < best_val_ridge: best_val_ridge, best_lam_ridge, best_w_ridge = v, lam, w
print("Ridge (GD): best λ =", best_lam_ridge, " | val MSE =", round(best_val_ridge, 6))

# Elastic Net
best_lam_en, best_ratio_en, best_val_en, best_w_en = None, None, 1e18, None
for ratio in [0.3, 0.5, 0.7]:
    for lam in lambda_list_l1:
        w = gd_elastic(Phi_train.values, y_train, lam, l1_ratio=ratio, lr=0.02, niter=20000)
        v = mse(Phi_val.values, y_val, w)
        if v < best_val_en: best_val_en, best_lam_en, best_ratio_en, best_w_en = v, lam, ratio, w
print("Elastic (GD): best λ =", best_lam_en, " | l1_ratio =", best_ratio_en, " | val MSE =", round(best_val_en, 6))

# quick sklearn comparison 
if best_lam_lasso is not None:
    sk_lasso = Lasso(alpha=best_lam_lasso, fit_intercept=False, max_iter=10000)
    sk_lasso.fit(Phi_train.values, y_train.ravel())
    print("Lasso (sklearn) val MSE:", round(mean_squared_error(y_val.ravel(), sk_lasso.predict(Phi_val.values)), 6))

sk_ridge = Ridge(alpha=best_lam_ridge, fit_intercept=False)
sk_ridge.fit(Phi_train.values, y_train.ravel())
print("Ridge  (sklearn) val MSE:", round(mean_squared_error(y_val.ravel(), sk_ridge.predict(Phi_val.values)), 6))

sk_en = ElasticNet(alpha=best_lam_en, l1_ratio=best_ratio_en, fit_intercept=False, max_iter=10000)
sk_en.fit(Phi_train.values, y_train.ravel())
print("Elastic(sklearn) val MSE:", round(mean_squared_error(y_val.ravel(), sk_en.predict(Phi_val.values)), 6))


Lasso (GD): best λ = 1.0  | val MSE = 139.677923
Ridge (GD): best λ = 0.1  | val MSE = 113.414032
Elastic (GD): best λ = 0.1  | l1_ratio = 0.7  | val MSE = 63.232482
Lasso (sklearn) val MSE: 27.388184
Ridge  (sklearn) val MSE: 56.481691
Elastic(sklearn) val MSE: 32.055884


Problem 3 (d)

In [8]:
from sklearn.metrics import r2_score

candidates = [
    ("lasso",   best_val_lasso, best_lam_lasso, best_w_lasso, None),
    ("ridge",   best_val_ridge, best_lam_ridge, best_w_ridge, None),
    ("elastic", best_val_en,    best_lam_en,    best_w_en,    best_ratio_en),
]
candidates.sort(key=lambda z: z[1])
winner, best_val_mse, lam_star, w_star, ratio_star = candidates[0]

print("Best model:", winner, "| val MSE:", round(best_val_mse, 6))

# test performance
yhat_test = np.dot(Phi_test.values, w_star)
test_mse = float(np.mean((yhat_test - y_test)**2))
test_r2  = float(r2_score(y_test.ravel(), yhat_test.ravel()))
print("Test MSE:", round(test_mse, 6), "| R^2:", round(test_r2, 6))

# refit winner on full data
Phi_full = pd.concat([Phi_train, Phi_val, Phi_test], axis=0)
y_full   = np.vstack([y_train, y_val, y_test])

if winner == "ridge":
    w_final = gd_ridge(Phi_full.values, y_full, lam_star, lr=0.02, niter=12000)
elif winner == "lasso":
    w_final = gd_lasso(Phi_full.values, y_full, lam_star, lr=0.02, niter=25000)
else:
    w_final = gd_elastic(Phi_full.values, y_full, lam_star, l1_ratio=ratio_star, lr=0.02, niter=25000)

assert np.isfinite(w_final).all()

print("Final model:", winner, "| λ =", lam_star, ("| l1_ratio = " + str(ratio_star) if winner=="elastic" else ""))
print("Number of weights:", w_final.shape[0])
print("First 10 weights:")
for i in range(min(10, w_final.shape[0])):
    print("w", i, "=", round(float(w_final[i,0]), 6))

Best model: elastic | val MSE: 63.232482
Test MSE: 32937.470386 | R^2: -65.047473


/var/folders/vw/c11105x93rdfqx32_1qmzfn40000gn/T/ipykernel_79488/2079171815.py:51: RuntimeWarning: invalid value encountered in multiply
  w_mid = w - lr * grad


Final model: elastic | λ = 0.1 | l1_ratio = 0.7
Number of weights: 11
First 10 weights:
w 0 = 0.0
w 1 = 7.862561676340847e+303
w 2 = 1.269478444438594e+301
w 3 = 2.2180115670081132e+303
w 4 = 5.669348035942454e+304
w 5 = -1.0284880703786672e+303
w 6 = 1.4461760743770974e+304
w 7 = 1.0693225253038282e+303
w 8 = 5.122796753376923e+302
w 9 = 4.692353899054133e+303
